In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [ ]:
import os
import sys
from pathlib import Path

src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
#os.environ["PYTHONPATH"] = src_dir

In [ ]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
import config
import pickle
from ray import tune
import matplotlib.pyplot as plt

load the gray-matter, and blood-vessel pathlengths and scattering counts

In [ ]:
gray_matter_mcs_pathlength_data = np.loadtxt(config.pl_dir / "gray_matter_mcs.txt")
blood_vessel_mcs_pathlength_data = np.loadtxt(config.pl_dir / "blood_vessel_pure_mcs.txt")

create MCS-Attenuation Object

# Forward Test

- Input tissue concentrations, and reconstruct spectra with every model
- compute squared average (over wavelength range) attenuation error per tissue and model

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "rb") as f:
    wavelengths, simulation_data = pickle.load(f)

In [ ]:
forward_results = np.empty(4, 6, len(wavelengths))

Interpolate path and scatter lengths for MBLL + MCS

In [ ]:
lengths = tuple()
for mcs_data in (gray_matter_mcs_pathlength_data, blood_vessel_mcs_pathlength_data):
    pathlength = np.interp(wavelengths, mcs_data[:, 0], mcs_data[:, 4])
    # notice the minus: mean scatterlength = mean pathlength - ((mean scattercount)/mu_s)
    scatterlength = -np.interp(wavelengths, mcs_data[:, 0], mcs_data[:, 5])
    scatterlength /= mcs_data[:, 2]
    scatterlength += pathlength
    lengths += (pathlength, scatterlength)

gray_matter_pathlength, gray_matter_scatterlength, blood_vessel_pathlength, blood_vessel_scatterlength = lengths